In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder

def prepare_inputs(X_train, X_test):
    oe = OrdinalEncoder()
    oe.fit(X_train.reshape(-1,1))
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc


def Label_Encode(X_train,X_test):
    LE = LabelEncoder()
    if type(X_train).__name__ == 'Series' :
        X_train_le = LE.fit_transform(X_train)
        X_test_le = LE.fit_transform(X_test)
    else :
        X_train_le = X_train.values.copy()
        X_test_le = X_test.values.copy()
        for i in range(X_train.shape[1]):
            X_train_le[:,i] = LE.fit_transform(X_train.values[:,i])
            X_test_le[:,i] = LE.fit_transform(X_test.values[:,i])
    return X_train_le , X_test_le




In [3]:
data=pd.read_csv('https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-04/train.csv')


data=data.drop(["Cabin","Age"],axis=1)
data=data.dropna()

data.head()
    

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,S


In [4]:
data_train, data_test = train_test_split( data, test_size=0.2, random_state=42)

###  1. Dùng các features: Sex, Pclass, và Embarked để xây dựng thuật toán Naive Bayes Multinomial. (4đ)

In [5]:
from sklearn.preprocessing import LabelEncoder
from collections import Counter # hàm Counter dùng để đếm bên trong mỗi phần tử khác nhau sẽ xuất hiện bao nhiêu lần

LE = LabelEncoder()

class MultinomialNaiveBayes :
    def __init__(self,Laplace_Smooth = False) :
        self.Laplace_Smooth = Laplace_Smooth
    def fit(self,X_train,y_train) :
        self.Class = np.unique(y_train)
        self.N = X_train.shape[0]
        
        self.M = 1
        self.X_train_LE = X_train.values.copy()
        
        if type(X_train).__name__ == 'DataFrame' :
            self.M = X_train.shape[1]
            for i in range(self.M):
                self.X_train_LE[:,i] = LE.fit_transform(X_train.values[:,i])
        else :
            self.X_train_LE = LE.fit_transform(X_train)

            
        self.prior = []
        for i in self.Class :
            self.prior.append(np.count_nonzero(y_train == i) / self.N)
        
    def predict(self,X_test):  
        self.X_test_LE = X_test.values.copy()
        if self.M == 1 :
            self.X_test_LE = LE.fit_transform(X_test)
        else :
            for i in range(self.M):
                self.X_test_LE[:,i] = LE.fit_transform(X_test.values[:,i])
            
        posterior = []
        for i in range(len(self.Class)) :
            class_num = np.count_nonzero(y_train == self.Class[i])
            for k in range(self.M):
                posterior_feature = []
                feature_prob = Counter(self.X_train_LE[:,k])
                if self.Laplace_Smooth == True :
                    z = len(np.unique(self.X_train_LE[:,k]))
                    for j in pd.DataFrame(self.X_test_LE).index : 
                        posterior_feature.append( (1 + feature_prob[self.X_test_LE[:,k][j]] ) / (class_num + z ) )
                else :
                     for j in pd.DataFrame(self.X_test_LE).index :
                        posterior_feature.append(feature_prob[self.X_test_LE[:,k][j]] / class_num )
                posterior.append(np.array(posterior_feature))
                
        self.posterior = np.array(posterior).T
        self.likelihood = []
        
        c = 1
        for i in range(self.M* len(self.Class), 0 , -self.M):
            self.likelihood.append(np.log( np.prod(self.posterior[:,i-self.M : i ],axis = 1) *self.prior[c] ) )
            c -= 1 
        self.likelihood = np.array(self.likelihood).T
        self.y_predict = self.likelihood.argmax(axis = 1)
        return self.y_predict
    def score(self,y_test) :
        return np.mean(self.y_predict == y_test)
        

In [6]:
X_train = data_train[['Sex','Pclass', 'Embarked']]
X_test = data_test[['Sex','Pclass', 'Embarked']]


y_train = data_train['Survived']
y_test = data_test['Survived']


model = MultinomialNaiveBayes(Laplace_Smooth = False)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)


print('Score của hàm tự build : {} '.format(model.score(y_test)))

from sklearn.naive_bayes import MultinomialNB

X_train_LE , X_test_LE = Label_Encode(X_train,X_test) # Hàm của thư viện cần encode trước khi sài
model_sklearn = MultinomialNB().fit(X_train_LE, y_train)
print('Score của hàm sklearn : {} '.format(model_sklearn.score(X_test_LE,y_test)))

# kiểm tra y_predict của hàm tự build và thư viện
y_predict == model_sklearn.predict(X_test_LE) 

Score của hàm tự build : 0.6123595505617978 
Score của hàm sklearn : 0.6123595505617978 


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

### 2. Dùng các features: SibSp,	Parch, và Fare để xây dựng thuật toán Naive Bayes Gaussian (Khỏi normalize). (4đ)

In [7]:
class GaussianNaiveBayes:
    def __init__(self):
        pass
    def fit(self,X_train,y_train):
        self.Class = np.unique(y_train)
        self.N = X_train.shape[0]
        
        self.prior = []
        for i in self.Class:
            self.prior.append(np.count_nonzero(y_train == i )/self.N)
            
        self.mean = []
        self.std = []
        for i in range(len(self.Class)):
            self.mean.append(np.mean(X_train[y_train == self.Class[i]], axis = 0))
            self.std.append(np.std(X_train[y_train == self.Class[i]], axis = 0))

    def predict(self,X_test): 
        gaussian = []
        likelihood = []
        for i in range(len(self.Class)) :
            gaussian.append(1/( np.sqrt(2 * np.pi) * self.std[i] ) * np.exp(-(X_test - self.mean[i])**2 /(2 * self.std[i]**2)))
            if type(X_test).__name__  == 'Series': # trường hợp dữ liệu chỉ có 1 cột
                likelihood.append(np.log(gaussian[i]*self.prior[i])) 
            else :                                 # trường hợp dữ liệu có nhiều cột
                likelihood.append(np.log(np.prod(gaussian[i],axis = 1) *self.prior[i] ))
        self.likelihood = np.array(likelihood).T
        self.y_predict = self.likelihood.argmax(axis = 1)
        return self.y_predict
    def score(self,y_test) :
        return np.mean(self.y_predict == y_test)
        

In [8]:
X_train = data_train[['SibSp','Parch', 'Fare']]
X_test = data_test[['SibSp','Parch', 'Fare']]


y_train = data_train['Survived']
y_test = data_test['Survived']


model = GaussianNaiveBayes()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)


print('Score của hàm tự build : {} '.format(model.score(y_test)))

from sklearn.naive_bayes import GaussianNB
model_sklearn = GaussianNB().fit(X_train, y_train)

print('Score của hàm sklearn : {} '.format(model_sklearn.score(X_test,y_test)))

# kiểm tra y_predict của hàm tự build và thư viện
y_predict == model_sklearn.predict(X_test) 

Score của hàm tự build : 0.6910112359550562 
Score của hàm sklearn : 0.6910112359550562 


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

### 3. Từ bộ dữ liệu liên minh hãy xây dựng thuật toán Naive Bayes bằng cách dùng tất cả features (mix cả Multinomial và Gaussian. Lưu ý: Khỏi chia tập train, test. Dự đoán thẳng trên tập train) (2đ)

In [9]:
#Chuẩn bị data cho câu 3
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/dinhvietcuong1996/Lab-MachineLearningCourse/master/Lab04/lienminh.csv")
df.head()

,killsDiff,minionsKilledDiff,wardPlacedDiff,firstBlood,heralds,dragons,teamWins
0,3,-2,13,blue,none,none,red
1,0,-66,0,red,red,red,red
2,-4,-17,0,red,none,blue,red
3,-1,-34,28,red,blue,none,red
4,0,-15,58,red,none,red,red


In [10]:
print('Shape của data ban đầu: ',df.shape)
# drop các giá trị 'none' trong data
df = df.mask(df.astype(object).eq('none'))
df.dropna(inplace = True)
print('Shape của data sau khi bỏ giá trị none :',df.shape)

Shape của data ban đầu:  (9879, 7)
Shape của data sau khi bỏ giá trị none : (2792, 7)


In [11]:
for i in range(df.shape[1]) :
    print('Cột {} có {} giá trị unique'.format(i+1,len(np.unique(df.iloc[:,i])) ))


Cột 1 có 30 giá trị unique
Cột 2 có 191 giá trị unique
Cột 3 có 199 giá trị unique
Cột 4 có 2 giá trị unique
Cột 5 có 2 giá trị unique
Cột 6 có 2 giá trị unique
Cột 7 có 2 giá trị unique


Ta thấy rằng ở 3 cột đầu thì dữ liệu liên tục nên ta sẽ sử dụng Gaussian cho 3 cột này, còn đối với 3 cột còn lại ta sẽ sử dụng Multinomial. Cột cuối cùng là Label.

In [12]:
X_train_continous = df.iloc[:, 0:3]
X_train_continous.head()

,killsDiff,minionsKilledDiff,wardPlacedDiff
1,0,-66,0
8,0,-51,1
18,1,-15,2
23,1,18,63
31,3,12,2


In [13]:
X_train_discrete = df.iloc[:,3:6]
X_train_discrete.head()
# X_train_discrete_LE = X_train_discrete.copy()
# for i in range(X_train_discrete.shape[1]):
#     X_train_discrete_LE.iloc[:,i] = LE.fit_transform(X_train_discrete.iloc[:,i])

# X_train_discrete_LE.head()

,firstBlood,heralds,dragons
1,red,red,red
8,red,red,red
18,blue,red,red
23,red,blue,blue
31,blue,blue,blue


In [14]:
y_train = df.iloc[:,6]


LE = LabelEncoder()
y_train = LE.fit_transform(y_train)
np.unique(y_train,return_counts = True)

(array([0, 1]), array([1402, 1390], dtype=int64))

In [15]:
class Gaussian_MultinomialNB:
    def __init__(self,Laplace_Smooth = False):
        self.Laplace_Smooth = Laplace_Smooth
        
    def fit(self,X_train_continous, X_train_discrete,y_train):
        
        
        self.X_train_continous = X_train_continous

        self.N = X_train_continous.shape[0] 
        self.y_train = LE.fit_transform(y_train)
        self.Class = np.unique(y_train)
        
        self.M2 = 1
        self.X_train_LE = X_train_discrete.copy()
        
        if type(X_train_discrete).__name__ == 'DataFrame' :
            self.M2 = X_train.shape[1]
            for i in range(self.M2):
                self.X_train_LE.iloc[:,i] = LE.fit_transform(X_train_discrete.iloc[:,i])
        else :
            self.X_train_LE = LE.fit_transform(X_train_discrete)

         
        self.prior = []
        for i in self.Class :
            self.prior.append(np.count_nonzero(y_train == i) / self.N)

    def predict(self,X_test_continous, X_test_discrete):        
        mean = []
        std = []
        posterior_1 = []
        for i in range(len(self.Class)) :
            mean.append(np.mean(self.X_train_continous[self.y_train == self.Class[i]],axis = 0))    
            std.append(np.std(self.X_train_continous[self.y_train == self.Class[i]], axis = 0))
            posterior_1.append( 1/(np.sqrt(2*  np.pi )* std[i])* np.exp(-(self.X_train_continous - mean[i] )**2 /(2* std[i]**2)) )

            
        self.X_test_LE = X_test_continous.copy()
        if type(X_test_discrete).__name__ == 'DataFrame' :
            for i in range(self.M2):
                self.X_test_LE.iloc[:,i] = LE.fit_transform(X_test_continous.iloc[:,i])
        else :
            self.X_test_LE = LE.fit_transform(X_test_LE)

        posterior_2 = []
        for i in range(len(self.Class)) :
            class_num = np.count_nonzero(self.y_train == self.Class[i])
            for k in range(self.M2) :
                posterior_feature = []
                feature_prob = Counter(self.X_train_LE.iloc[:,k])
                if self.Laplace_Smooth == True :
                    z = len(np.unique(self.X_test_LE.iloc[:,k]))
                    for j in pd.DataFrame(self.X_test_LE).index:
                        posterior_feature.append( (1 + feature_prob[pd.DataFrame(self.X_test_LE).iloc[:,k][j]])/ (class_num + z) )
                else :
                    for j in pd.DataFrame(self.X_test_LE).index:
                        posterior_feature.append(feature_prob[pd.DataFrame(self.X_test_LE).iloc[:,k][j]]/class_num )
                posterior_2.append(np.array(posterior_feature))
        posterior_2 = np.array(posterior_2).T

        posterior = []
        likelihood = []
        for i in range(len(self.Class)):
            posterior.append(np.concatenate((posterior_1[i], posterior_2[:,i:i+self.M2]) , axis = 1))
            likelihood.append(np.log(np.prod(posterior[i], axis = 1) * self.prior[i]))
        likelihood = np.array(likelihood).T
        self.y_predict = likelihood.argmax(axis = 1)
        return self.y_predict
    def score(self,y_test):
        return np.mean(self.y_predict == y_test)
    
        


In [16]:
models = Gaussian_MultinomialNB(Laplace_Smooth= True)
models.fit(X_train_continous,X_train_discrete, y_train)
y_predict = models.predict(X_train_continous,X_train_discrete)

print('Score : {} '.format(models.score(y_train)))

Score : 0.6962750716332379 
